In [ ]:
def remove_eta_lines(run_number):
    input_file_path = f'D:\\thesis_data\\notebooks\\model_runs\\{run_number}\\log.txt'
    
    with open(input_file_path, 'r') as f:
        lines = f.readlines()

    # Filter out lines containing "- ETA: "
    lines = [line for line in lines if "- ETA: " not in line]

    # Save the modified content back to the same file
    with open(input_file_path, 'w') as f:
        f.writelines(lines)

In [ ]:
def divide_range(a, b, x):
    x = x+1
    a = a -0.01
    b = b + 0.01

    if round(a,0) == 46:
        if x > 11:
            x = 11
            print("Warning: 10 is the highest number of divisions possible for latitudes. So the number of divsions is set to maximum that is 10.")
    
    if round(a,0)==10:
        if x > 15:
            x = 15
            print("Warning: 14 is the highest number of divisions possible for longitudes. So the number of divsions is set to maximum that is 14.")

    if x < 2:
        raise ValueError("Number of divisions must be at least 2")

    step = (b - a) / (x - 1)  # Calculate the step size for equal divisions
    result = [round(a + i * step, 2) for i in range(x)]

    return result

In [ ]:
def pad_month_fraction_array(month_fraction_array):
    # Define the indices where 1 should be added
    indices_to_pad = [0, 1, 2, 11]
    
    # Create the padded array by inserting 1 at specified indices
    month_fraction_array_padded = month_fraction_array.copy()
    for idx in indices_to_pad:
        month_fraction_array_padded.insert(idx, 1)
    
    return month_fraction_array_padded

In [ ]:
def get_dates(ndsi_ds):
    time_ndsi = ndsi_ds["time"].values
    df_time = pd.DataFrame({"date": time_ndsi})
    df_time[["date"]]= df_time[['date']].astype(str)
    df_time['date'] = pd.to_datetime(df_time['date'])
    df_time['date'] = df_time['date'] + pd.Timedelta(hours=12)
    selected_dates = df_time['date'].values
    return selected_dates


In [ ]:
def get_filtered_dates_for_ndsi(percentage_limit=0.35, pixel_interpolation_limit=50):
    nan_interpolation_df = pd.read_csv(r"D:\thesis_data\random_csv_output\number_of_nans_and_interpolations_for_each_date.csv")
    nan_interpolation_df = nan_interpolation_df.iloc[:,1:] 

    nan_interpolation_df["Number of original pixels"] = 140 - nan_interpolation_df["Number of NaNs"]
    nan_interpolation_df['Number of interpolations done'] = nan_interpolation_df["Number of NaNs"] - nan_interpolation_df["Number of NaNs after interpolation"]
    nan_interpolation_df['Percentage to original'] = nan_interpolation_df["Number of interpolations done"]/(140 - nan_interpolation_df["Number of NaNs"])
    
    nan_interpolation_df["check"] = np.where((nan_interpolation_df["Number of interpolations done"]) >= pixel_interpolation_limit, 1, 0 )
    nan_interpolation_df["check2"] = np.where((nan_interpolation_df["Percentage to original"]) >= percentage_limit, 1, 0 )

    nan_interpolation_df_filtered = nan_interpolation_df[(nan_interpolation_df["check"] == 0) & (nan_interpolation_df["check2"] == 0)]
    return nan_interpolation_df_filtered["Date"].values, nan_interpolation_df

In [ ]:

    
def to_array(data, index_climate_parameter):
    array = []
    n_time_steps = data[0].shape[0]
    n_lat, n_lon = data[0].shape[1], data[0].shape[2]
    data1 = data[index_climate_parameter]
    for b in range(n_time_steps):
        data2=data1[b]
        for c in range(n_lat):
            data3=data2[c]
            for d in range(n_lon):
                data4=data3[d]
                array.append(data4)
    array = np.array(array)
    return array


In [ ]:
def get_min_max(df, parameter_name):
    param_data = df[df['parameter'] == parameter_name]
    if not param_data.empty:
        min_val = param_data['min'].values[0]
        max_val = param_data['max'].values[0]
        return min_val, max_val
    else:
        return None, None 

In [ ]:

def to_array_ndsi(data):
    array = []
    n_time_steps = data.shape[0]
    n_lat, n_lon = data.shape[1], data.shape[2]
    data1 = data
    for b in range(n_time_steps):
        data2=data1[b]
        for c in range(n_lat):
            data3=data2[c]
            for d in range(n_lon):
                data4=data3[d]
                array.append(data4)
    array = np.array(array)
    return array

In [ ]:
import math

def self_defineed_r2(y_test, y_pred):
    mean_true = np.mean(y_test)
    mean_pred = np.mean(y_pred)
    numerator = np.sum((y_test - mean_true)*(y_pred-mean_pred))
    denominator1 = math.sqrt(np.sum((y_test - mean_true)**2))
    denominator2 = math.sqrt(np.sum((y_pred - mean_pred)**2))

    self_defined_r2 =  (numerator/(denominator1*denominator2))**2
    return self_defined_r2

In [1]:
def pbias(y_test, y_pred):
    num = np.sum(y_test - y_pred)
    denom = np.sum(y_test)
    return num*100/denom


In [ ]:
def nse(y_pred, y_test):
    mean_observed = np.mean(y_test)
    numerator = np.sum((y_test - y_pred)**2)
    denominator = np.sum((y_test - mean_observed)**2)
    nse_value = 1 - (numerator / denominator)
    return nse_value

In [ ]:
def calculate_n_rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_true = np.mean(y_test)
    n_rmse = rmse / mean_true
    return n_rmse